# Unit 12: Case Study 

Allen Ansari, Chris Ballenger, Shantanu Godbole, Chad Madding

DS 7333 Quantifying the World

July 25, 2020

---

Given the following paper: https://arxiv.org/pdf/1402.4735.pdf 

Build a replica Neural Network with the paper’s architecture using Tensorflow. 

If possible, begin to train on the data located here: https://archive.ics.uci.edu/ml/datasets/HIGGS 

How close can you get to the original results? 

To facilitate quicker training, you may increase the batch size temporarily (this has a small impact on the final result but can speed your calculations significantly). You do not need to train a final result using the paper’s parameters; only the code for your model is required in your final submission. 

Include in your report: 
Based on the class notes and discussion, suggest improvements to the procedure. 
What are standard practices now versus when this paper was written? 
What kind of improvements do they provide? 
How would you quantify if your result duplicated the papers? 

 

---

## Table of Contents
1. [Abstract & Introduction](#Abstract)
2. [Data](#Data)
3. [Description of Neural Network in the Paper](#Description-of-Neural-Network-in-the-Paper)
4. [Re-Creation of Neural Network in Keras](#Re-Creation-of-Neural-Network-in-Keras)
 - [Observations and Workarounds](#Observations-and-Workarounds) 
 - [Neural Network Code](#Neural-Network-Code) 
5. [Enhancements to the Network](#Enhancements-to-the-Network) 
6. [Conclusion](#Conclusion)
7. [Appendix](#Appendix)


---

## Abstract 

We will be carrying out a classification problem to distinguish between a signal process that produces Higgs bosons and a background process that does not. Our information is based on outdated code, so in the process, we will reproduce results with more updated information. 

## Introduction 

### What are standard practices now versus when this paper was written? 

In the paper “Searching for Exotic Particles in High-Energy Physics with Deep Learning,” published in 2014, the authors looked at using deep learning models to assist in classifying “exotic particle(s)” that result from particle collisions. Their paper focused on using the Higgs Boson data collected from the Large Hadron Collider’s detectors, and their study focused on improving past research that used ‘shallow’ machine learning models. According to the paper, these models have a limited capacity to learn complex non-linear functions. 

Their findings ended with Deep Learning outperforming the standard at the time, “shallow” models. The original research and building of the model used libraries standard in 2014. The original code, implemented through pylearn2, is no longer supported. 

In this project, we will use the same dataset and the Keras Python library with a TensorFlow backend to replicate their original work using more modern approaches. 

 

### Keras 

Keras is a neural network library written in Python and can run on top of TensorFlow and other existing toolkits. It is touted for its’ ease of use and can be up and running with just a few lines of code while still being customizable enough for layer by layer programming. 

### TensorFlow 

TensorFlow is a free and open-source platform developed by the Google Brain Team. It is a software library for dataflow and differentiable programming across a range of tasks and used for machine learning applications such as neural networks. (TensorFlow, n.d.) 


[Back to Top](#Table-of-Contents)
____

## Dataset 

The data has been produced using Monte Carlo simulations. The first 21 features (columns 2-22) are kinematic properties measured by the particle detectors in the accelerator. The last seven features are functions of the first 21 features; these are high-level features derived by physicists to help discriminate between the two classes. 

[Back to Top](#Table-of-Contents)
____

## Description-of-Neural-Network-in-the-Paper:

* Five-layered neural network with 300 hidden units in each layer

* The Activation function that was used in all layers and all nodes is 'tanh'

* Weights were initialized from a normal distribution with zero mean and standard deviation 0.1 in the first layer, 0.001 in the output layer, and 0.05 all other hidden layers

* Weight decay coefficient of 1 × 10^-5 was set for all the layers

* Stocastic Gradient Descent was used as the optimizer

* The initial learning rate was set to 0.05 and thereafter the learning rate was decayed by a factor of 1.0000002 every batch until it reached to the minimum learning rate of 10^-6

* A momentum term increased linearly over the first 200 epochs from 0.9 to 0.99, at which point it remained constant

* Gradient computations were made on mini-batches of size 100.

* Training ended when the momentum had reached its maximum value and the minimum error on the validation set (500,000 examples) had not decreased by more than a factor of 0.00001 over 10 epochs.

* The Validation size was 500,000

* The Neural Network was created using the Theano library


[Back to Top](#Table-of-Contents)
____


## Re-Creation-of-Neural-Network-in-Keras 

#### Observations-and-Workarounds

 1. There was no option that was found to dynamically adjust the momentum in keras, therefore the standard SGD momentum was used 
 
 2. Also, as a result of no momentum change, the early stopping condition that involves momentum could not be used
 
 3. To replicate the lr decay, the 'LR Scdeduler' that is provided by Keras callback functions was earlier used but that decays the learning rate per Epoch which did not fit our requirement
 
 4. To adress the above problem, a class `LearningRateBatchScheduler` was inherited from `(tf.keras.callbacks.Callback)`. The on_batch_end method was used to make sure that the change in Learning Rate happens at the end of the batch. The print statement in the code (which is now commented) was used to verify the same. We referred to the [Stackoverflow Link](https://stackoverflow.com/questions/52277003/how-to-implement-exponentially-decay-learning-rate-in-keras-by-following-the-glo) for this.
 
 5. The Weight Decay were set by using l2 function in the kernel regularizer 
 
 6. For setting the initial weights in the different laters, per the normal discribution and different standard deviations mentioned in the paper, `kernel_initializer` parameter in the dense layer was used   
 
 7. We used the Validation set that is 30% of the total dataset, just to speed up the training by reducing the number of records in the test set
 
 8. Batch size in the paper was 100, but again for speeding up the training we will be using bath size of 500
 
 9. The minimum change in loss for early stopping, was increased to '0.0001' , from '0.00001' and the patience was decreased to 2 for stopping the training earlier as the Network was still getting decent AUC 
 
 10. As per [this Medium article](https://towardsdatascience.com/learning-rate-schedules-and-adaptive-learning-rate-methods-for-deep-learning-2c8f433990d1#:~:text=The%20mathematical%20form%20of%20time,t%20is%20the%20iteration%20number.) the formula for Exponential lr decay is `lr = lr0 * e^(−kt)` where lr, k are hyperparameters and t is the iteration number. We tried 1.0000002 as the rate that was mentioned in the paper. The LR was being floored to minimum in just 3 batches. To facilitate gradual decrease of the Learning rate we used 0.0000002 as the decay rate. However the use of class `LearningRateBatchScheduler_exp` that implements this formula made the AUC to be stuck around 0.75. We tried linear decay instead of exponential decay that is implemented in the class `LearningRateBatchScheduler_linear`, which gave us the AUC that is comparable to what was acchieved in the paper
 
 11. The Paper mentions the use of training layers using Autoencoder and then using the layers tarined in the autoencoder (pre-trained layers) in the classification Neural Netowrk. This can be potentially tried as a further imporvement to the basic NN with Dropout.
 
 12. We have scaled to a mean of 0 and standard diev of 1.

[Back to Top](#Table-of-Contents)
____

#### Neural-Network-Code

In [38]:
### Imports 

import pandas as pd
import numpy as np

In [2]:
## Read the data 

df_data = pd.read_hdf('data.h5', 'table', where=['index>2'])

np_data = np.array(df_data.iloc[:,1:])
lables = np.array(df_data.iloc[:,0])

In [3]:
df_data.tail()

,label,lepton pt,lepton eta,lepton phi,missing energy magnitude,missing energy phi,jet 1 pt,jet 1 eta,jet 1 phi,jet 1 b-tag,...,jet 4 eta,jet 4 phi,jet 4 b-tag,m_jj,m_jjj,m_lv,m_jlv,m_bb,m_wbb,m_wwbb
10999995,1.0,1.160156,1.013672,0.108643,1.495117,-0.537598,2.341797,-0.839844,1.320312,0.000000,...,-0.097046,1.190430,3.101562,0.822266,0.766602,1.001953,1.061523,0.836914,0.860352,0.772461
10999996,1.0,0.618164,-1.012695,1.110352,0.940918,-0.379150,1.004883,0.348633,-1.678711,2.173828,...,-0.217041,1.048828,3.101562,0.826660,0.989746,1.029297,1.199219,0.891602,0.938477,0.865234
10999997,1.0,0.700684,0.774414,1.520508,0.847168,0.211182,1.095703,0.052460,0.024551,2.173828,...,1.584961,1.713867,0.000000,0.337402,0.845215,0.987793,0.883301,1.888672,1.153320,0.931152
10999998,0.0,1.177734,0.117798,-1.277344,1.864258,-0.584473,0.998535,-1.264648,1.276367,0.000000,...,1.399414,-1.313477,0.000000,0.838867,0.882812,1.201172,0.939453,0.339600,0.759277,0.719238
10999999,0.0,0.464355,-0.337158,0.229004,0.954590,-0.868652,0.429932,-0.271240,-1.251953,2.173828,...,-1.652344,-0.586426,0.000000,0.752441,0.740723,0.986816,0.664062,0.576172,0.541504,0.517578


In [4]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
np_data_scaled = sc.fit_transform(np_data)

In [5]:
from sklearn.model_selection import train_test_split

x_train_split, x_test_split, y_train_split, y_test_split = train_test_split(np_data_scaled, lables, test_size = 0.30, random_state = 7)

In [6]:
### Imports 

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Activation, Dropout
from tensorflow.keras.utils import normalize, to_categorical
from sklearn.metrics import roc_curve, auc
from tensorflow.keras import optimizers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras import initializers
from tensorflow.keras.layers import LeakyReLU

In [7]:
### Nueral Network Parameters

batch_size = 500 #Batch Size modified from what was mentioned in the paper to speed up the training process
weights_decay = 0.00001

lr = 0.15
min_lr = 0.000001
lr_decay_factor = 0.0000002
lr_decay_factor_e = 0.0000002


initializer_input = tf.keras.initializers.RandomNormal(mean = 0, stddev = 0.1)
initializer_output = tf.keras.initializers.RandomNormal(mean = 0, stddev = 0.001)
initializer_hidden_layers = tf.keras.initializers.RandomNormal(mean = 0, stddev = 0.05)


In [8]:
class LearningRateBatchScheduler_linear(tf.keras.callbacks.Callback):
  
    def __init__(self, update_freq=None):
        self._update_freq = update_freq
        self.lr =lr

    def on_batch_end(self, batch, logs=None):
        if self._update_freq and batch % self._update_freq != 0:
            return
        # To avoid divergence, limit the value range.
        
        if self.lr <= min_lr:
            self.lr = min_lr
        else:    
            self.lr = self.lr - lr_decay_factor
        tf.keras.backend.set_value(self.model.optimizer.lr, self.lr)
#         print("lr:",self.lr)

In [9]:
class LearningRateBatchScheduler_exp(tf.keras.callbacks.Callback):
  
    def __init__(self, update_freq=None):
        self._update_freq = update_freq
        self.lr = lr
        self.iteration = 1

    def on_batch_end(self, batch, logs=None):
        if self._update_freq and batch % self._update_freq != 0:
            return
        # To avoid divergence, limit the value range.
        
        if self.lr <= min_lr:
            self.lr = min_lr
        else:    
            self.lr = self.lr * np.exp(-lr_decay_factor_e * self.iteration)
            self.iteration = self.iteration + 1
        tf.keras.backend.set_value(self.model.optimizer.lr, self.lr)
#         print("lr:",self.lr)

In [10]:
## Early Stopping 

earlyStopping = EarlyStopping(monitor='loss', mode='min', min_delta = 0.0001 , patience = 2)

callbacks_a = [ LearningRateBatchScheduler_linear(), earlyStopping]


In [11]:
model = Sequential()

model.add(tf.keras.Input(shape=(28,)))
# model.add(Dropout(0.5))

model.add(Dense(300,activation = 'tanh',kernel_regularizer=l2(weights_decay), bias_regularizer=l2(weights_decay),kernel_initializer = initializer_hidden_layers))

model.add(Dense(300,activation = 'tanh',kernel_initializer = initializer_hidden_layers ,kernel_regularizer=l2(weights_decay), bias_regularizer=l2(weights_decay)))

model.add(Dense(300,activation = 'tanh', kernel_initializer = initializer_hidden_layers, kernel_regularizer=l2(weights_decay), bias_regularizer=l2(weights_decay)))

model.add(Dense(1,activation = 'sigmoid',kernel_initializer = initializer_output))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 300)               8700      
_________________________________________________________________
dense_1 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_2 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 301       
Total params: 189,601
Trainable params: 189,601
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Compiling the ANN
model.compile(optimizer = optimizers.SGD(), loss = 'binary_crossentropy', metrics = ['accuracy',tf.keras.metrics.AUC()])

In [13]:
# Fitting the ANN
model.fit(x_train_split, y_train_split, batch_size = batch_size, validation_data=(x_test_split,y_test_split), epochs = 1000,callbacks = callbacks_a)


Train on 7699997 samples, validate on 3300000 samples
Epoch 1/1000
7699997/7699997 [==============================] - 60s 8us/sample - loss: 0.5590 - accuracy: 0.7108 - auc: 0.7855 - val_loss: 0.5246 - val_accuracy: 0.7369 - val_auc: 0.8183
Epoch 2/1000
7699997/7699997 [==============================] - 58s 8us/sample - loss: 0.5185 - accuracy: 0.7413 - auc: 0.8223 - val_loss: 0.5135 - val_accuracy: 0.7443 - val_auc: 0.8266
Epoch 3/1000
7699997/7699997 [==============================] - 59s 8us/sample - loss: 0.5101 - accuracy: 0.7470 - auc: 0.8291 - val_loss: 0.5064 - val_accuracy: 0.7493 - val_auc: 0.8322
Epoch 4/1000
7699997/7699997 [==============================] - 56s 7us/sample - loss: 0.5045 - accuracy: 0.7508 - auc: 0.8335 - val_loss: 0.5023 - val_accuracy: 0.7523 - val_auc: 0.8358
Epoch 5/1000
7699997/7699997 [==============================] - 56s 7us/sample - loss: 0.5005 - accuracy: 0.7536 - auc: 0.8367 - val_loss: 0.4977 - val_accuracy: 0.7554 - val_auc: 0.8389
Epoch 6/100

#### In the paper they had a complete AUC of 0.876. We were able to acchieve a comparable AUC of 0.8595

[Back to Top](#Table-of-Contents)
____

## Enhancements to the Network
  

#### 1. As discussed in the class, SGD does the job but takes a lot of time to converge. Also the use of a fixed momentum limited our performance in this case. There are many other optimization algorithms that are known to perform faster and better, such as Adam, Nadam, Adagard, Adamax. All these optimazition algorithms are in some way based on SDG, but with variations to make the convergence faster. 'AD' in the most of the above mentioned algorithms stand for 'Adaptive' Momentum. For example: Adam keeps the average of 'n' exponentially decaying average of the past gradients and adjust the learning rate accordingly.


##### Trying to run the network with relu as the activation function and Adadelta just for 10 Epochs to see if it converges any faster 

In [23]:
model1 = Sequential()

model1.add(tf.keras.Input(shape=(28,)))

model1.add(Dense(300,activation = 'relu',kernel_regularizer=l2(weights_decay), bias_regularizer=l2(weights_decay),kernel_initializer = initializer_hidden_layers))
# model.add(Dropout(0.2))

model1.add(Dense(300,activation = 'relu',kernel_initializer = initializer_hidden_layers ,kernel_regularizer=l2(weights_decay), bias_regularizer=l2(weights_decay)))
# model.add(Dropout(0.2))

model1.add(Dense(300,activation = 'relu', kernel_initializer = initializer_hidden_layers, kernel_regularizer=l2(weights_decay), bias_regularizer=l2(weights_decay)))
# model.add(Dropout(0.2))

model1.add(Dense(1,activation = 'sigmoid',kernel_initializer = initializer_output))


model1.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 300)               8700      
_________________________________________________________________
dense_9 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_10 (Dense)             (None, 300)               90300     
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 301       
Total params: 189,601
Trainable params: 189,601
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Compiling the ANN
model1.compile(optimizer = optimizers.Adadelta(), loss = 'binary_crossentropy', metrics = ['accuracy',tf.keras.metrics.AUC()])

In [25]:
# Fitting the ANN
model1.fit(x_train_split, y_train_split, batch_size = batch_size, validation_data=(x_test_split,y_test_split), epochs = 10)


Train on 7699997 samples, validate on 3300000 samples
Epoch 1/10
7699997/7699997 [==============================] - 59s 8us/sample - loss: 0.6966 - accuracy: 0.5301 - auc_2: 0.5271 - val_loss: 0.6953 - val_accuracy: 0.5305 - val_auc_2: 0.5827
Epoch 2/10
7699997/7699997 [==============================] - 56s 7us/sample - loss: 0.6932 - accuracy: 0.5431 - auc_2: 0.5996 - val_loss: 0.6903 - val_accuracy: 0.5627 - val_auc_2: 0.6179
Epoch 3/10
7699997/7699997 [==============================] - 56s 7us/sample - loss: 0.6843 - accuracy: 0.5810 - auc_2: 0.6267 - val_loss: 0.6763 - val_accuracy: 0.5964 - val_auc_2: 0.6422
Epoch 4/10
7699997/7699997 [==============================] - 57s 7us/sample - loss: 0.6648 - accuracy: 0.6099 - auc_2: 0.6572 - val_loss: 0.6529 - val_accuracy: 0.6234 - val_auc_2: 0.6734
Epoch 5/10
7699997/7699997 [==============================] - 56s 7us/sample - loss: 0.6439 - accuracy: 0.6344 - auc_2: 0.6850 - val_loss: 0.6368 - val_accuracy: 0.6424 - val_auc_2: 0.6945
E

##### The Model Training on Epoch 1 starts with low Accuracy and AUC, but the adaptable momentum benifits are definately seen as the training progresses 

[Back to Top](#Table-of-Contents)
____

#### 2. Number of nodes in the network: It has been mentioned that various combinations of number of nodes and layers were tried, however a deeper neural network could be worth trying to see if accuracy increases with increasing number of nodes. 


##### We will try to train the network that has 500 Nodes in each layer and has one extra layer to see if it makes any difference 

In [50]:

callbacks_model2 = [earlyStopping]

model2 = Sequential()

model2.add(tf.keras.Input(shape=(28,)))

model2.add(Dense(500,activation = 'relu',kernel_regularizer=l2(weights_decay), bias_regularizer=l2(weights_decay),kernel_initializer = initializer_hidden_layers))

model2.add(Dense(500,activation = 'relu',kernel_initializer = initializer_hidden_layers ,kernel_regularizer=l2(weights_decay), bias_regularizer=l2(weights_decay)))

model2.add(Dense(500,activation = 'relu', kernel_initializer = initializer_hidden_layers, kernel_regularizer=l2(weights_decay), bias_regularizer=l2(weights_decay)))

model2.add(Dense(500,activation = 'relu', kernel_initializer = initializer_hidden_layers, kernel_regularizer=l2(weights_decay), bias_regularizer=l2(weights_decay)))

model2.add(Dense(1,activation = 'sigmoid',kernel_initializer = initializer_output))

# Compiling the ANN
model2.compile(optimizer = optimizers.Adadelta(), loss = 'binary_crossentropy', metrics = ['accuracy',tf.keras.metrics.AUC()])

# # Fitting the ANN
# model2.fit(x_train_split, y_train_split, batch_size = batch_size, validation_data=(x_test_split,y_test_split), epochs = 1000, callbacks = callbacks_model2)

model2.summary()


Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_40 (Dense)             (None, 500)               14500     
_________________________________________________________________
dense_41 (Dense)             (None, 500)               250500    
_________________________________________________________________
dense_42 (Dense)             (None, 500)               250500    
_________________________________________________________________
dense_43 (Dense)             (None, 500)               250500    
_________________________________________________________________
dense_44 (Dense)             (None, 1)                 501       
Total params: 766,501
Trainable params: 766,501
Non-trainable params: 0
_________________________________________________________________


![title](Capture.png)

##### We ran the above code. The training loss and increase in AUC looked promising in the initial iterations, however for a time Val accuracy was around 0.8335. Therefore we had to stop the training. Threre was a early stopping method set as a callback, however as the loss was slightly decreasing, the training kept on running

[Back to Top](#Table-of-Contents)
____

#### 3. Cyclic Learning Rate: The learning rate usually goes from high to low in the decay process, hoping that the local minimum was already traversed when the Learning Rate was high enough, but it might be the case that model is being stuck in a local minimum. Cyclin learning rate oscillates in the cycles of highs and lows. Therefore even if the model gets stuck in the local minimum, the upcoming cycle of hyigh Learning rate can get the model out of the local minimum and the model can go towards global minimum.  
  

##### We will try to implement Traingular Learning Rate over Epochs to see if the convergence can be any faster 

In [32]:
class LearningRateBatchScheduler_traingular(tf.keras.callbacks.Callback):
  
    def __init__(self, update_freq=None):
        self._update_freq = update_freq
        self.lr = lr
        self.iteration = 1

    def on_batch_end(self, batch, logs=None):
        if self._update_freq and batch % self._update_freq != 0:
            return
        # To avoid divergence, limit the value range.
        
        if self.lr <= min_lr:
            self.lr = lr
        else:    
            self.lr = self.lr - lr_decay_factor
        tf.keras.backend.set_value(self.model.optimizer.lr, self.lr)
#         print("lr:",self.lr)

In [34]:
callbacks_model3 = [ LearningRateBatchScheduler_traingular(), earlyStopping]

model3 = Sequential()

model3.add(tf.keras.Input(shape=(28,)))

model3.add(Dense(300,activation = 'relu',kernel_initializer = initializer_hidden_layers ,kernel_regularizer=l2(weights_decay), bias_regularizer=l2(weights_decay)))
# model.add(Dropout(0.2))

model3.add(Dense(300,activation = 'relu', kernel_initializer = initializer_hidden_layers, kernel_regularizer=l2(weights_decay), bias_regularizer=l2(weights_decay)))
# model.add(Dropout(0.2))

model3.add(Dense(300,activation = 'relu', kernel_initializer = initializer_hidden_layers, kernel_regularizer=l2(weights_decay), bias_regularizer=l2(weights_decay)))
# model.add(Dropout(0.2))

model3.add(Dense(1,activation = 'sigmoid',kernel_initializer = initializer_output))

# Compiling the ANN
model3.compile(optimizer = optimizers.Adam(), loss = 'binary_crossentropy', metrics = ['accuracy',tf.keras.metrics.AUC()])

# Fitting the ANN
model3.fit(x_train_split, y_train_split, batch_size = 1000, validation_data=(x_test_split,y_test_split), epochs = 1000, callbacks = callbacks_model2)


Train on 7699997 samples, validate on 3300000 samples
Epoch 1/1000
7699997/7699997 [==============================] - 32s 4us/sample - loss: 0.5075 - accuracy: 0.7484 - auc_7: 0.8308 - val_loss: 0.4881 - val_accuracy: 0.7623 - val_auc_7: 0.8466
Epoch 2/1000
7699997/7699997 [==============================] - 31s 4us/sample - loss: 0.4846 - accuracy: 0.7649 - auc_7: 0.8491 - val_loss: 0.4831 - val_accuracy: 0.7667 - val_auc_7: 0.8510
Epoch 3/1000
7699997/7699997 [==============================] - 31s 4us/sample - loss: 0.4790 - accuracy: 0.7692 - auc_7: 0.8537 - val_loss: 0.4792 - val_accuracy: 0.7690 - val_auc_7: 0.8544
Epoch 4/1000
7699997/7699997 [==============================] - 31s 4us/sample - loss: 0.4758 - accuracy: 0.7717 - auc_7: 0.8564 - val_loss: 0.4755 - val_accuracy: 0.7719 - val_auc_7: 0.8569
Epoch 5/1000
7699997/7699997 [==============================] - 31s 4us/sample - loss: 0.4737 - accuracy: 0.7734 - auc_7: 0.8581 - val_loss: 0.4737 - val_accuracy: 0.7736 - val_auc_7

##### The Combination of Adam and the Traingular learning rate got the Validation AUC in the range of 0.86 relative faster, but did not go over 0.86

[Back to Top](#Table-of-Contents)
____

## Conclusion

* Despite of some changes in the libraries, we were able to get very close to the AUC in the Paper 

* Adding the Dropout layer actually made our model give a slightly less AUC

* The Enhancements to the model that differ from the paper - such as Adding more nodes and layers, using different optimization algorithms such as Adam and Admax and using triangular learning rate, gave us very close results however the combination of Adam with the traingular learning rate reached the AUC of 0.86 plus in just 8 Epochs